In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import plotly.express as px


In [3]:
# load data
inspections = pd.read_csv('inspections.csv')
invoices = pd.read_csv('invoices.csv')
quotes = pd.read_csv('quotes.csv')

In [4]:
# fix data types
inspections['date'] = pd.to_datetime(inspections['date'])
invoices['date'] = pd.to_datetime(invoices['date'])
quotes['proposed_date'] = pd.to_datetime(quotes['proposed_date'])
quotes['create_date'] = pd.to_datetime(quotes['create_date'])
quotes['convert_date'] = pd.to_datetime(quotes['convert_date'])

/var/folders/pp/dqmbfdr53hn607y3m8cg2gp00000gp/T/ipykernel_38153/2360018396.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  inspections['date'] = pd.to_datetime(inspections['date'])
/var/folders/pp/dqmbfdr53hn607y3m8cg2gp00000gp/T/ipykernel_38153/2360018396.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  invoices['date'] = pd.to_datetime(invoices['date'])
/var/folders/pp/dqmbfdr53hn607y3m8cg2gp00000gp/T/ipykernel_38153/2360018396.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  quotes['proposed_date'] = pd.to_datetime(quotes['proposed_date'])
/var/folders/pp/dqmb

In [5]:
# fix more data types
quotes['total'] = quotes['total'].str.replace('$', '')
quotes['total'] = quotes['total'].str.replace(',', '')
quotes['total'] = quotes['total'].str.replace('(', '')
quotes['total'] = quotes['total'].str.replace(')', '')
quotes['total'] = quotes['total'].astype(float)
invoices['total'] = invoices['total'].str.replace('$', '')
invoices['total'] = invoices['total'].str.replace(',', '')
invoices['total'] = invoices['total'].astype(float)

In [6]:
# fill missing values in proposed_date with create_date
quotes['proposed_date'] = quotes.apply(
    lambda row: row['create_date'] if row['proposed_date'].year < 2000 else row['proposed_date'], axis=1)

In [7]:
# create a new column 'sold' which has a value of 1 if the quote was sold, 0 otherwise
quotes['sold'] = quotes['label'].apply(lambda x: 1 if x == 'Converted To Work' else 0)

In [8]:
# drop create_date column
quotes = quotes.drop(columns=['create_date'])

In [9]:
# monthly sales performance
# count of quotes sent per month
quotes['proposed_month'] = quotes['proposed_date'].dt.to_period('M')
monthly_performance = quotes.groupby('proposed_month').size().reset_index(name='count')
quotes['convert_month'] = quotes['convert_date'].dt.to_period('M')
monthly_performance.rename(columns={'proposed_month': 'month', 'count':'quotes_sent'}, inplace=True)
monthly_performance['quotes_sold'] = quotes[quotes['sold'] == 1].groupby('convert_month').size().reset_index(name='count')['count']
monthly_performance['conversion_rate'] = monthly_performance['quotes_sold'] / monthly_performance['quotes_sent']
monthly_performance['sales'] = quotes[quotes['sold'] == 1].groupby('convert_month')['total'].sum().reset_index()['total']
monthly_performance['efficiency'] = monthly_performance['sales'] / monthly_performance['quotes_sent']

In [10]:
monthly_performance.head(25)

,month,quotes_sent,quotes_sold,conversion_rate,sales,efficiency
0,2018-02,5,1.0,0.200000,100.00,20.000000
1,2018-03,11,2.0,0.181818,7528.00,684.363636
2,2018-04,70,35.0,0.500000,97515.30,1393.075714
3,2018-05,43,61.0,1.418605,202275.76,4704.087442
4,2018-06,37,70.0,1.891892,270191.98,7302.485946
5,2018-07,19,59.0,3.105263,252765.45,13303.444737
6,2018-08,89,53.0,0.595506,197621.65,2220.467978
7,2018-09,159,66.0,0.415094,185829.80,1168.740881
8,2018-10,153,96.0,0.627451,337100.69,2203.272484
9,2018-11,150,82.0,0.546667,293153.15,1954.354333


In [11]:
# add monthly revenue and inspections
invoices['invoice_month'] = invoices['date'].dt.to_period('M')
monthly_performance['revenue'] = invoices.groupby('invoice_month')['total'].sum().reindex(monthly_performance['month']).reset_index(drop=True)
inspections['month'] = inspections['date'].dt.to_period('M')
monthly_performance['inspections'] = inspections.groupby('month').size().reindex(monthly_performance['month']).reset_index(drop=True)
monthly_performance['quote_rate'] = monthly_performance['quotes_sent'] / monthly_performance['inspections']
monthly_performance['last_month_quotes_sent'] = monthly_performance['quotes_sent'].shift(1)
monthly_performance.set_index('month', inplace=True)
monthly_performance.index = pd.to_datetime(monthly_performance.index.to_timestamp())

In [12]:
monthly_performance.head(25)

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent
month,,,,,,,,,
2018-02-01,5,1.0,0.200000,100.00,20.000000,NaN,NaN,NaN,NaN
2018-03-01,11,2.0,0.181818,7528.00,684.363636,NaN,1.0,11.000000,5.0
2018-04-01,70,35.0,0.500000,97515.30,1393.075714,74422.35,169.0,0.414201,11.0
2018-05-01,43,61.0,1.418605,202275.76,4704.087442,153938.85,189.0,0.227513,70.0
2018-06-01,37,70.0,1.891892,270191.98,7302.485946,200025.82,218.0,0.169725,43.0
2018-07-01,19,59.0,3.105263,252765.45,13303.444737,214718.05,214.0,0.088785,37.0
2018-08-01,89,53.0,0.595506,197621.65,2220.467978,204239.96,222.0,0.400901,19.0
2018-09-01,159,66.0,0.415094,185829.80,1168.740881,200367.95,189.0,0.841270,89.0
2018-10-01,153,96.0,0.627451,337100.69,2203.272484,300283.61,285.0,0.536842,159.0


In [13]:
# remove all 2018 months from montly performance
monthly_performance = monthly_performance[monthly_performance.index.year != 2018]

In [14]:
monthly_performance.tail()

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent
month,,,,,,,,,
2024-09-01,1177,434.0,0.368734,2529838.33,2149.395353,2514816.75,1150.0,1.023478,1296.0
2024-10-01,1360,503.0,0.369853,3052943.12,2244.811118,2961509.44,1418.0,0.959097,1177.0
2024-11-01,963,424.0,0.440291,2679493.25,2782.443666,2554753.36,1002.0,0.961078,1360.0
2024-12-01,606,284.0,0.468647,1882557.16,3106.529967,1894702.97,656.0,0.923780,963.0
2025-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,606.0


In [15]:
# remove 2025 from monthly performance
monthly_performance = monthly_performance[monthly_performance.index.year != 2025]

In [16]:
# Plotly Bar Chart
fig = px.bar(
    monthly_performance.reset_index(),
    x='month',
    y='sales',
    title='Monthly Sales',
    labels={'month': 'Month', 'sales': 'Total'}
)

# Customize Layout (for x-axis rotation and better visualization)
fig.update_layout(
    xaxis_tickangle=-90,
    height=600,
    width=1000,
    xaxis_title='Month',
    yaxis_title='Total'
)

fig.show()
fig.write_html('monthly_sales.html')

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [40]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# SARIMA hyperparameters
sales_sarima_model = SARIMAX(monthly_performance['sales'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sales_sarima_result = sales_sarima_model.fit()

# Forecast range (end of 2026)
forecast_steps = 24  # Forecast for 24 months (end of 2026)

# Forecast the values
forecast = sales_sarima_result.get_forecast(steps=forecast_steps)
forecast_df = forecast.summary_frame()

# Prepare forecast dates
forecast_index = pd.date_range(monthly_performance.index[-1], periods=forecast_steps+1, freq='M')[1:]
forecast_df['date'] = forecast_index

# Plot with Plotly
fig = go.Figure()

# Plot observed sales
fig.add_trace(go.Scatter(
    x=monthly_performance.index,
    y=monthly_performance['sales'],
    mode='lines',
    name='Sales'
))

# Plot forecasted sales
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange', dash='dash')
))

# Confidence interval (shaded region)
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_upper'],
    fill=None,
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Upper Bound'
))

fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_lower'],
    fill='tonexty',
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Lower Bound'
))

# Layout customization
fig.update_layout(
    title='Monthly Sales with Seasonality Forecast',
    xaxis_title='Date',
    yaxis_title='Sales (Millions)',
    height=600,
    width=900,
    legend_title='Legend'
)

fig.show()
fig.write_html('monthly_sales_forecast.html')

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.14021D+01    |proj g|=  1.03458D-01

At iterate    5    f=  1.13636D+01    |proj g|=  1.06376D-02

At iterate   10    f=  1.13627D+01    |proj g|=  3.92467D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     12     14      1     0     0   3.552D-06   1.136D+01
  F =   11.362740666903857     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


In [18]:
warnings.filterwarnings('default')
print(forecast_df)

sales               mean        mean_se  mean_ci_lower  mean_ci_upper  \
2025-01-01  2.580280e+06  295084.987392   2.001924e+06   3.158636e+06   
2025-02-01  2.778098e+06  321716.776990   2.147545e+06   3.408652e+06   
2025-03-01  2.966703e+06  330667.837927   2.318606e+06   3.614800e+06   
2025-04-01  2.949233e+06  336134.105417   2.290422e+06   3.608043e+06   
2025-05-01  2.835149e+06  340668.764378   2.167451e+06   3.502848e+06   
2025-06-01  2.858364e+06  344899.693882   2.182373e+06   3.534355e+06   
2025-07-01  3.214432e+06  349005.909353   2.530393e+06   3.898471e+06   
2025-08-01  3.365509e+06  353041.996772   2.673559e+06   4.057458e+06   
2025-09-01  3.203629e+06  357025.620904   2.503872e+06   3.903387e+06   
2025-10-01  3.303928e+06  360963.209734   2.596453e+06   4.011403e+06   
2025-11-01  2.849837e+06  364857.734703   2.134729e+06   3.564945e+06   
2025-12-01  2.316039e+06  368711.110869   1.593378e+06   3.038699e+06   
2026-01-01  3.038878e+06  417126.895996   2.221324e

In [19]:
# calculate the total sales 2024
total_sales_2024 = monthly_performance['sales']['2024-01':'2024-12'].sum()
print('Total sales in 2024: ', total_sales_2024)

Total sales in 2024:  29961833.96


In [20]:
# calculate the total sales for 2025
total_sales_2025 = forecast_df['mean'].loc['2025-01-01':'2025-12-01'].sum()
print('Total Sales for 2025:', total_sales_2025)

Total Sales for 2025: 35221201.04019658


In [21]:
# calculate the total sales for 2026
total_sales_2026 = forecast_df['mean'].loc['2026-01-01':'2026-12-01'].sum()
print('Total sales for 2026:', total_sales_2026)

Total sales for 2026: 40592851.342426695


In [22]:
# bar chart with yearly sales
yearly_sales = monthly_performance.resample('YE').sum()
# add total sales for 2025 and 2026
yearly_sales.loc['2025'] = total_sales_2025
yearly_sales.loc['2026'] = total_sales_2026

In [23]:
yearly_sales.index = pd.to_datetime(yearly_sales.index)
yearly_sales['year'] = yearly_sales.index.year

In [24]:
import plotly.express as px
import pandas as pd

# Define colors
yearly_sales['year'] = yearly_sales.index.year
yearly_sales['colors'] = ['blue' if year < 2025 else '#ffcc80' for year in yearly_sales.index.year]

# Plotly bar plot
fig = px.bar(
    yearly_sales,
    x='year',
    y='sales',
    color='colors',
    title='Yearly Sales',
    labels={'sales': 'Total', 'year': 'Year'},  # Tooltip label customization
    hover_data={'year': True, 'sales': True, 'colors': False}  # Hide 'colors' in tooltip
)

# Customize layout
fig.update_layout(
    xaxis=dict(tickangle=0),
    yaxis_title='Total',
    xaxis_title='Year',
    showlegend=False,
    height=600,
    width=1000
)

fig.show()
fig.write_html('yearly_sales.html')

/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/opt/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [25]:
# monthly inspections
# Plotly Bar Chart
fig = px.bar(
    monthly_performance.reset_index(),
    x='month',
    y='inspections',
    title='Monthly Inspections',
    labels={'month': 'Month', 'inspections': 'Inspections'}
)

# Customize Layout (for x-axis rotation and better visualization)
fig.update_layout(
    xaxis_tickangle=-90,
    height=600,
    width=1000,
    xaxis_title='Month',
    yaxis_title='Inspections'
)

fig.show()
fig.write_html('monthly_inspections.html')

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [41]:
# monthly inspections forecast
# SARIMA hyperparameters
inspections_sarima_model = SARIMAX(monthly_performance['inspections'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
inspections_sarima_result = inspections_sarima_model.fit()

# Forecast range (end of 2026)
forecast_steps = 24  # Forecast for 24 months (end of 2026)

# Forecast the values
forecast = inspections_sarima_result.get_forecast(steps=forecast_steps)
forecast_df = forecast.summary_frame()

# Prepare forecast dates
forecast_index = pd.date_range(monthly_performance.index[-1], periods=forecast_steps+1, freq='M')[1:]
forecast_df['date'] = forecast_index

# Plot with Plotly
fig = go.Figure()

# Plot observed inspections
fig.add_trace(go.Scatter(
    x=monthly_performance.index,
    y=monthly_performance['inspections'],
    mode='lines',
    name='Inspections'
))

# Plot forecasted inspections
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange', dash='dash')
))

# Confidence interval (shaded region)
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_upper'],
    fill=None,
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Upper Bound'
))

fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_lower'],
    fill='tonexty',
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Lower Bound'
))

# Layout customization
fig.update_layout(
    title='Monthly Inspections with Seasonality Forecast',
    xaxis_title='Date',
    yaxis_title='Inspections',
    height=600,
    width=900,
    legend_title='Legend'
)

fig.show()
fig.write_html('monthly_inspections_forecast.html')

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.16602D+00    |proj g|=  4.07279D-01

At iterate    5    f=  5.02561D+00    |proj g|=  1.36188D-02

At iterate   10    f=  5.02293D+00    |proj g|=  4.05102D-02

At iterate   15    f=  5.01989D+00    |proj g|=  4.39506D-03

At iterate   20    f=  5.01746D+00    |proj g|=  1.26814D-02

At iterate   25    f=  5.01712D+00    |proj g|=  1.13972D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     27     30      1     0     0   

 This problem is unconstrained.


In [27]:
# calculate the total inspections for 2025
total_inspections_2025 = forecast_df['mean'].loc['2025-01-01':'2025-12-01'].sum()
print('Total inspections for 2025:', total_inspections_2025)

# calculate the total inspections for 2026
total_inspections_2026 = forecast_df['mean'].loc['2026-01-01':'2026-12-01'].sum()
print('Total inspections for 2026:', total_inspections_2026)

# bar chart with yearly inspections
yearly_inspections = monthly_performance.resample('YE').sum()
# add total inspections for 2025 and 2026
yearly_inspections.loc['2025'] = total_inspections_2025
yearly_inspections.loc['2026'] = total_inspections_2026

yearly_inspections.index = pd.to_datetime(yearly_inspections.index)
yearly_inspections['year'] = yearly_inspections.index.year

# Define colors
yearly_inspections['year'] = yearly_inspections.index.year
yearly_inspections['colors'] = ['blue' if year < 2025 else '#ffcc80' for year in yearly_inspections.index.year]

# Plotly bar plot
fig = px.bar(
    yearly_inspections,
    x='year',
    y='inspections',
    color='colors',
    title='Yearly Inspections',
    labels={'inspections': 'Total', 'year': 'Year'},  # Tooltip label customization
    hover_data={'year': True, 'inspections': True, 'colors': False}  # Hide 'colors' in tooltip
)

# Customize layout
fig.update_layout(
    xaxis=dict(tickangle=0),
    yaxis_title='Total',
    xaxis_title='Year',
    showlegend=False,
    height=600,
    width=1000
)

fig.show()
fig.write_html('yearly_inspections.html')

Total inspections for 2025: 14826.121834946918
Total inspections for 2026: 16853.124795503933


/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/opt/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [28]:
# monthly revenue
# Plotly Bar Chart
fig = px.bar(
    monthly_performance.reset_index(),
    x='month',
    y='revenue',
    title='Monthly Revenue',
    labels={'month': 'Month', 'revenue': 'Revenue'}
)

# Customize Layout (for x-axis rotation and better visualization)
fig.update_layout(
    xaxis_tickangle=-90,
    height=600,
    width=1000,
    xaxis_title='Month',
    yaxis_title='Revenue'
)

fig.show()
fig.write_html('monthly_revenue.html')

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [42]:
# monthly revenue forecast
# SARIMA hyperparameters
revenue_sarima_model = SARIMAX(monthly_performance['revenue'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
revenue_sarima_result = revenue_sarima_model.fit()

# Forecast range (end of 2026)
forecast_steps = 24  # Forecast for 24 months (end of 2026)

# Forecast the values
forecast = revenue_sarima_result.get_forecast(steps=forecast_steps)
forecast_df = forecast.summary_frame()

# Prepare forecast dates
forecast_index = pd.date_range(monthly_performance.index[-1], periods=forecast_steps+1, freq='M')[1:]
forecast_df['date'] = forecast_index

# Plot with Plotly
fig = go.Figure()

# Plot observed revenue
fig.add_trace(go.Scatter(
    x=monthly_performance.index,
    y=monthly_performance['revenue'],
    mode='lines',
    name='Revenue'
))

# Plot forecasted revenue
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange', dash='dash')
))

# Confidence interval (shaded region)
fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_upper'],
    fill=None,
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Upper Bound'
))

fig.add_trace(go.Scatter(
    x=forecast_df['date'],
    y=forecast_df['mean_ci_lower'],
    fill='tonexty',
    mode='lines',
    line_color='rgba(255,165,0,0.3)',
    name='Lower Bound'
))

# Layout customization
fig.update_layout(
    title='Monthly Revenue with Seasonality Forecast',
    xaxis_title='Date',
    yaxis_title='Revenue',
    height=600,
    width=900,
    legend_title='Legend'
)

fig.show()
fig.write_html('monthly_revenue_forecast.html')

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.11456D+01    |proj g|=  1.00211D-01

At iterate    5    f=  1.11153D+01    |proj g|=  8.56586D-03

At iterate   10    f=  1.11081D+01    |proj g|=  2.62722D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     13     16      1     0     0   7.083D-05   1.111D+01
  F =   11.108094402753736     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


In [30]:
# calculate the total revenue for 2025
total_revenue_2025 = forecast_df['mean'].loc['2025-01-01':'2025-12-01'].sum()
print('Total revenue for 2025:', total_revenue_2025)

# calculate the total revenue for 2026
total_revenue_2026 = forecast_df['mean'].loc['2026-01-01':'2026-12-01'].sum()
print('Total revenue for 2026:', total_revenue_2026)

# bar chart with yearly revenue
yearly_revenue = monthly_performance.resample('YE').sum()
# add total revenue for 2025 and 2026
yearly_revenue.loc['2025'] = total_revenue_2025
yearly_revenue.loc['2026'] = total_revenue_2026

yearly_revenue.index = pd.to_datetime(yearly_revenue.index)
yearly_revenue['year'] = yearly_revenue.index.year

# Define colors
yearly_revenue['year'] = yearly_revenue.index.year
yearly_revenue['colors'] = ['blue' if year < 2025 else '#ffcc80' for year in yearly_revenue.index.year]

# Plotly bar plot
fig = px.bar(
    yearly_revenue,
    x='year',
    y='revenue',
    color='colors',
    title='Yearly Revenue',
    labels={'revenue': 'Total', 'year': 'Year'},  # Tooltip label customization
    hover_data={'year': True, 'revenue': True, 'colors': False}  # Hide 'colors' in tooltip
)

# Customize layout
fig.update_layout(
    xaxis=dict(tickangle=0),
    yaxis_title='Total',
    xaxis_title='Year',
    showlegend=False,
    height=600,
    width=1000
)

fig.show()
fig.write_html('yearly_revenue.html')

Total revenue for 2025: 32689835.166102573
Total revenue for 2026: 37342188.2765636


/opt/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/opt/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [31]:
yearly_sales

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent,year,colors
month,,,,,,,,,,,
2019-12-31,1.749000e+03,8.870000e+02,6.236656e+00,3.742929e+06,2.645781e+04,3.660184e+06,2.237000e+03,9.397078e+00,1.775000e+03,2019,blue
2020-12-31,1.209000e+03,1.802000e+03,2.140808e+01,7.314158e+06,8.793191e+04,6.736588e+06,3.948000e+03,4.205136e+00,1.254000e+03,2020,blue
2021-12-31,8.330000e+02,3.236000e+03,1.391636e+02,1.501107e+07,6.463777e+05,1.419963e+07,8.205000e+03,1.329946e+00,9.050000e+02,2021,blue
2022-12-31,7.388000e+03,3.943000e+03,8.589047e+01,1.919668e+07,4.121749e+05,1.855228e+07,9.333000e+03,9.264061e+00,6.787000e+03,2022,blue
2023-12-31,1.155400e+04,4.503000e+03,4.712567e+00,2.441817e+07,2.539333e+04,2.360077e+07,1.163700e+04,1.190596e+01,1.162700e+04,2023,blue
2024-12-31,1.273700e+04,5.089000e+03,4.848120e+00,2.996183e+07,2.865106e+04,2.808853e+07,1.294100e+04,1.178844e+01,1.266200e+04,2024,blue
2025-01-01,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,3.522120e+07,2025,#ffcc80
2026-01-01,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,4.059285e+07,2026,#ffcc80


In [32]:
yearly_revenue

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent,year,colors
month,,,,,,,,,,,
2019-12-31,1.749000e+03,8.870000e+02,6.236656e+00,3.742929e+06,2.645781e+04,3.660184e+06,2.237000e+03,9.397078e+00,1.775000e+03,2019,blue
2020-12-31,1.209000e+03,1.802000e+03,2.140808e+01,7.314158e+06,8.793191e+04,6.736588e+06,3.948000e+03,4.205136e+00,1.254000e+03,2020,blue
2021-12-31,8.330000e+02,3.236000e+03,1.391636e+02,1.501107e+07,6.463777e+05,1.419963e+07,8.205000e+03,1.329946e+00,9.050000e+02,2021,blue
2022-12-31,7.388000e+03,3.943000e+03,8.589047e+01,1.919668e+07,4.121749e+05,1.855228e+07,9.333000e+03,9.264061e+00,6.787000e+03,2022,blue
2023-12-31,1.155400e+04,4.503000e+03,4.712567e+00,2.441817e+07,2.539333e+04,2.360077e+07,1.163700e+04,1.190596e+01,1.162700e+04,2023,blue
2024-12-31,1.273700e+04,5.089000e+03,4.848120e+00,2.996183e+07,2.865106e+04,2.808853e+07,1.294100e+04,1.178844e+01,1.266200e+04,2024,blue
2025-01-01,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,3.268984e+07,2025,#ffcc80
2026-01-01,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,3.734219e+07,2026,#ffcc80


In [33]:
yearly_inspections

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent,year,colors
month,,,,,,,,,,,
2019-12-31,1749.000000,887.000000,6.236656,3.742929e+06,26457.813036,3.660184e+06,2237.000000,9.397078,1775.000000,2019,blue
2020-12-31,1209.000000,1802.000000,21.408075,7.314158e+06,87931.914178,6.736588e+06,3948.000000,4.205136,1254.000000,2020,blue
2021-12-31,833.000000,3236.000000,139.163554,1.501107e+07,646377.728616,1.419963e+07,8205.000000,1.329946,905.000000,2021,blue
2022-12-31,7388.000000,3943.000000,85.890465,1.919668e+07,412174.859859,1.855228e+07,9333.000000,9.264061,6787.000000,2022,blue
2023-12-31,11554.000000,4503.000000,4.712567,2.441817e+07,25393.332201,2.360077e+07,11637.000000,11.905958,11627.000000,2023,blue
2024-12-31,12737.000000,5089.000000,4.848120,2.996183e+07,28651.060372,2.808853e+07,12941.000000,11.788444,12662.000000,2024,blue
2025-01-01,14826.121835,14826.121835,14826.121835,1.482612e+04,14826.121835,1.482612e+04,14826.121835,14826.121835,14826.121835,2025,#ffcc80
2026-01-01,16853.124796,16853.124796,16853.124796,1.685312e+04,16853.124796,1.685312e+04,16853.124796,16853.124796,16853.124796,2026,#ffcc80


In [34]:
monthly_performance

,quotes_sent,quotes_sold,conversion_rate,sales,efficiency,revenue,inspections,quote_rate,last_month_quotes_sent
month,,,,,,,,,
2019-01-01,137,79.0,0.576642,310320.10,2265.110219,299900.81,180.0,0.761111,146.0
2019-02-01,152,67.0,0.440789,259325.00,1706.085526,271567.81,185.0,0.821622,137.0
2019-03-01,151,73.0,0.483444,321048.72,2126.150464,270408.39,174.0,0.867816,152.0
2019-04-01,146,67.0,0.458904,273658.59,1874.373904,344261.44,177.0,0.824859,151.0
2019-05-01,165,68.0,0.412121,272369.13,1650.722000,330279.66,190.0,0.868421,146.0
...,...,...,...,...,...,...,...,...,...
2024-08-01,1296,510.0,0.393519,3065502.44,2365.356821,2631526.28,1265.0,1.024506,1107.0
2024-09-01,1177,434.0,0.368734,2529838.33,2149.395353,2514816.75,1150.0,1.023478,1296.0
2024-10-01,1360,503.0,0.369853,3052943.12,2244.811118,2961509.44,1418.0,0.959097,1177.0


In [35]:
monthly_revenue = monthly_performance['revenue']

In [36]:
quarterly_revenue = monthly_revenue.resample('Q').sum()
quarterly_revenue

/var/folders/pp/dqmbfdr53hn607y3m8cg2gp00000gp/T/ipykernel_38153/547873638.py:1: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



month
2019-03-31     841877.01
2019-06-30     926727.48
2019-09-30     995633.57
2019-12-31     895945.87
2020-03-31    1092887.38
2020-06-30    1252180.77
2020-09-30    1979107.98
2020-12-31    2412411.92
2021-03-31    2637605.06
2021-06-30    3218514.96
2021-09-30    4146131.28
2021-12-31    4197377.18
2022-03-31    4180208.57
2022-06-30    4466621.79
2022-09-30    5152007.23
2022-12-31    4753441.89
2023-03-31    5240930.90
2023-06-30    5815991.24
2023-09-30    6541235.71
2023-12-31    6002614.98
2024-03-31    5838646.65
2024-06-30    7283139.57
2024-09-30    7555780.20
2024-12-31    7410965.77
Freq: QE-DEC, Name: revenue, dtype: float64

In [44]:
revyear = yearly_revenue['revenue']
revyear

month
2019-12-31    3.660184e+06
2020-12-31    6.736588e+06
2021-12-31    1.419963e+07
2022-12-31    1.855228e+07
2023-12-31    2.360077e+07
2024-12-31    2.808853e+07
2025-01-01    3.268984e+07
2026-01-01    3.734219e+07
Name: revenue, dtype: float64

In [45]:
# calculate percent revenue growth per year in revyear
revyearpct = revyear.pct_change()
revyearpct

month
2019-12-31         NaN
2020-12-31    0.840505
2021-12-31    1.107837
2022-12-31    0.306533
2023-12-31    0.272123
2024-12-31    0.190153
2025-01-01    0.163814
2026-01-01    0.142318
Name: revenue, dtype: float64

In [47]:
print(yearly_sales['sales'])

month
2019-12-31    3.742929e+06
2020-12-31    7.314158e+06
2021-12-31    1.501107e+07
2022-12-31    1.919668e+07
2023-12-31    2.441817e+07
2024-12-31    2.996183e+07
2025-01-01    3.522120e+07
2026-01-01    4.059285e+07
Name: sales, dtype: float64


In [48]:
# create a table with revyear and yearly_sales['sales] and yearly_inspections['inspections']
revenue_table = pd.concat([revyear, yearly_sales['sales'], yearly_inspections['inspections']], axis=1)
revenue_table.columns = ['revenue', 'sales', 'inspections']
revenue_table

,revenue,sales,inspections
month,,,
2019-12-31,3.660184e+06,3.742929e+06,2237.000000
2020-12-31,6.736588e+06,7.314158e+06,3948.000000
2021-12-31,1.419963e+07,1.501107e+07,8205.000000
2022-12-31,1.855228e+07,1.919668e+07,9333.000000
2023-12-31,2.360077e+07,2.441817e+07,11637.000000
2024-12-31,2.808853e+07,2.996183e+07,12941.000000
2025-01-01,3.268984e+07,3.522120e+07,14826.121835
2026-01-01,3.734219e+07,4.059285e+07,16853.124796


In [55]:
revenue_table.drop(columns=['inspections', 'sales'], inplace=True)

In [56]:
revenue_table

,revenue
month,
2019-12-31,3.660184e+06
2020-12-31,6.736588e+06
2021-12-31,1.419963e+07
2022-12-31,1.855228e+07
2023-12-31,2.360077e+07
2024-12-31,2.808853e+07
2025-01-01,3.268984e+07
2026-01-01,3.734219e+07


In [57]:
revenue_table['+10%'] = revenue_table['revenue'] * 1.1
revenue_table['-10%'] = revenue_table['revenue'] * 0.9

In [58]:
revenue_table

,revenue,+10%,-10%
month,,,
2019-12-31,3.660184e+06,4.026202e+06,3.294166e+06
2020-12-31,6.736588e+06,7.410247e+06,6.062929e+06
2021-12-31,1.419963e+07,1.561959e+07,1.277967e+07
2022-12-31,1.855228e+07,2.040751e+07,1.669705e+07
2023-12-31,2.360077e+07,2.596085e+07,2.124070e+07
2024-12-31,2.808853e+07,3.089739e+07,2.527968e+07
2025-01-01,3.268984e+07,3.595882e+07,2.942085e+07
2026-01-01,3.734219e+07,4.107641e+07,3.360797e+07


In [60]:
# change revenue_table index to column
revenue_table.reset_index(inplace=True)
revenue_table

,month,revenue,+10%,-10%
0,2019-12-31,3.660184e+06,4.026202e+06,3.294166e+06
1,2020-12-31,6.736588e+06,7.410247e+06,6.062929e+06
2,2021-12-31,1.419963e+07,1.561959e+07,1.277967e+07
3,2022-12-31,1.855228e+07,2.040751e+07,1.669705e+07
4,2023-12-31,2.360077e+07,2.596085e+07,2.124070e+07
5,2024-12-31,2.808853e+07,3.089739e+07,2.527968e+07
6,2025-01-01,3.268984e+07,3.595882e+07,2.942085e+07
7,2026-01-01,3.734219e+07,4.107641e+07,3.360797e+07


In [61]:
# change revenue_table['month'] to year
revenue_table['year'] = revenue_table['month'].dt.year
revenue_table

,month,revenue,+10%,-10%,year
0,2019-12-31,3.660184e+06,4.026202e+06,3.294166e+06,2019
1,2020-12-31,6.736588e+06,7.410247e+06,6.062929e+06,2020
2,2021-12-31,1.419963e+07,1.561959e+07,1.277967e+07,2021
3,2022-12-31,1.855228e+07,2.040751e+07,1.669705e+07,2022
4,2023-12-31,2.360077e+07,2.596085e+07,2.124070e+07,2023
5,2024-12-31,2.808853e+07,3.089739e+07,2.527968e+07,2024
6,2025-01-01,3.268984e+07,3.595882e+07,2.942085e+07,2025
7,2026-01-01,3.734219e+07,4.107641e+07,3.360797e+07,2026


In [62]:
revenue_table.drop(columns=['month'], inplace=True)
revenue_table

,revenue,+10%,-10%,year
0,3.660184e+06,4.026202e+06,3.294166e+06,2019
1,6.736588e+06,7.410247e+06,6.062929e+06,2020
2,1.419963e+07,1.561959e+07,1.277967e+07,2021
3,1.855228e+07,2.040751e+07,1.669705e+07,2022
4,2.360077e+07,2.596085e+07,2.124070e+07,2023
5,2.808853e+07,3.089739e+07,2.527968e+07,2024
6,3.268984e+07,3.595882e+07,2.942085e+07,2025
7,3.734219e+07,4.107641e+07,3.360797e+07,2026


In [63]:
# make 'year' the first column in revenue_table
revenue_table = revenue_table[['year', 'revenue', '+10%', '-10%']]
revenue_table

,year,revenue,+10%,-10%
0,2019,3.660184e+06,4.026202e+06,3.294166e+06
1,2020,6.736588e+06,7.410247e+06,6.062929e+06
2,2021,1.419963e+07,1.561959e+07,1.277967e+07
3,2022,1.855228e+07,2.040751e+07,1.669705e+07
4,2023,2.360077e+07,2.596085e+07,2.124070e+07
5,2024,2.808853e+07,3.089739e+07,2.527968e+07
6,2025,3.268984e+07,3.595882e+07,2.942085e+07
7,2026,3.734219e+07,4.107641e+07,3.360797e+07


In [65]:
# remove years 2019-2024 from revenue_table
revenue_table = revenue_table[revenue_table['year'] > 2024]
revenue_table

,year,revenue,+10%,-10%
6,2025,3.268984e+07,3.595882e+07,2.942085e+07
7,2026,3.734219e+07,4.107641e+07,3.360797e+07


In [72]:
revenue_table.rename(columns={'revenue': 'forecast'}, inplace=True)

In [77]:
import plotly.graph_objects as go

# Create a table with capitalized column names
fig = go.Figure(data=[go.Table(
    header=dict(values=[col.upper() for col in revenue_table.columns],
                fill_color='tan',
                align='left'),
    cells=dict(values=[revenue_table['year'], 
                       revenue_table['forecast'].map('${:,.2f}'.format), 
                       revenue_table['+10%'].map('${:,.2f}'.format), 
                       revenue_table['-10%'].map('${:,.2f}'.format)],
               fill_color='wheat',
               align='left'))
])

# Update layout
fig.update_layout(title='Revenue Table')

# Show the figure
fig.show()

In [78]:
revenue_table

,year,forecast,+10%,-10%
6,2025,3.268984e+07,3.595882e+07,2.942085e+07
7,2026,3.734219e+07,4.107641e+07,3.360797e+07


In [79]:
# change revenue_table values to currency
revenue_table['forecast'] = revenue_table['forecast'].map('${:,.2f}'.format)
revenue_table['+10%'] = revenue_table['+10%'].map('${:,.2f}'.format)
revenue_table['-10%'] = revenue_table['-10%'].map('${:,.2f}'.format)
revenue_table

,year,forecast,+10%,-10%
6,2025,"$32,689,835.17","$35,958,818.68","$29,420,851.65"
7,2026,"$37,342,188.28","$41,076,407.10","$33,607,969.45"
